# Simulador Agentes Montacargas que ordenan un almacen

In [1]:
# Multiagent system library
import agentpy as ap

# Math libraries
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

## Definición del Modelo

In [2]:
# Parámetros del modelo
parameters = {
    'densidad_cajas': 100, # Número de cajas
    'size': 20, # Altura y Longitud del grid
    'steps': 100,
    'montacargas': 5,
}

# Posibles movimientos de un montacargas
MOVIMIENTOS = [(-1, 0), # atras
               (1, 0), # enfrente
               (0, -1), # derecha
               (0, 1), # izquierda
               (-1, -1), # atras derecha
               (-1, 1), # atras izquierda
               (1, -1), # enfrente derecha
               (1, 1), # enfrente izquierda
               ]

In [3]:
class AlmacenModel(ap.Model):
     
     def setup(self):
          """ Definimos 2 cosas importantes:
               1. Cuántos montacargas vamos a crear en el modelo.
               2. La cantidad de cajas a ser ordenadas por los montacargas. 
          """
          
          # Calculamos el número de cajas dado un valor de densidad.
          n_cajas = int(self.p['densidad_cajas'])
          
          # Creamos 2 listas de agentes, una lista para cajas y otra lista de montacargas.
          montacargas = self.montacargas = ap.AgentList(self, self.p['montacargas'])
          cajas = self.cajas = ap.AgentList(self, n_cajas)

          # Creamos un ambiente tipo grid (almacen).
          self.almacen = ap.Grid(self, # la instancia del model
                                [self.p.size] * 2, # tamaño del grid (m x n)
                                torus = False,
                                track_empty = True, # tracks las celdas vacías
                                check_border = True, # asegura que el agente no se salga de los bordes
                                )
          
          # Definimos las posiciones iniciales de los montacargas
          posiciones_montacargas = [(1,1), (18,1), (1, 18), (18,18), (11, 11)]

          # Agregamos los agentes cajas y robots al grid en posición aleatoria.
          # ¿Cómo distinguir que uno es `montacarga` y otro es `caja`? => esto es
          #  importante porque cuando buscamos vecinos de un montacargas, nos
          #  debemos cerciorar que buscamos `cajas` y no otro `montacargas`. 
          
          self.almacen.add_agents(montacargas,
                                  positions = posiciones_montacargas,
                                  random = True,
                                  empty = True
                                  )
          
          self.almacen.add_agents(cajas, # iterable de agentes a agregar
                                  positions = None, # posiciones de los agentes
                                  random = True, # escogemos posiciones aleatorias
                                  empty = True # escoje las posiciones vacías
                                  )

          # Inicializamos una variable dinámica para los agentes
          # Condicion 0: Caja Desordenada
          # Condicion 1: Caja Ordenada
          # Condicion 2: Es Montacargas
          self.cajas.condicion = 0
          self.montacargas.condicion = 2

     def step(self):

          # Contamos las cajas desordenadas en el modelo
          cajas_desordenadas = self.cajas.select(self.cajas.condicion == 0)

          # Seleccionamos los montacargas en movimiento.
          montacargas_en_movimiento = self.montacargas.select(self.montacargas.condicion == 2)

          # Para cada montacarga en estado de movimiento, vemos si tiene cajas
          # vecinas a una distancia de 1 (en cada dirección, diagonal incluída).
          for montacarga in montacargas_en_movimiento:
               for celda_vecina in self.almacen.neighbors(agent=montacarga, distance=1):
                    if celda_vecina.condicion == 0: # si tiene condicion de caja desordenada
                         celda_vecina.condicion = 1 # agregamos condicion de caja ordenada
                         # break
               
               # Movemos al agente a una nueva posible posición
               self.almacen.move_by(montacarga, # instancia del agente que moveremos
                                    random.choice(MOVIMIENTOS) # cambio relativo de posición
                                   )

          # Detenemos simulación si no hay cajas desordenadas
          if len(cajas_desordenadas) == 0:
               self.stop()

     def end(self):
          
          # Documentamos el número de cajas ordenadas
          cajas_ordenadas = len(self.cajas.select(self.cajas.condicion == 1))
          self.report('Porcentaje de cajas ordenadas', cajas_ordenadas / len(self.cajas))

In [6]:
def animation_plot(model, ax):

     # Grid de atributos para cajas y montacargas
     attr_grid = model.almacen.attr_grid('condicion')

     # Mapa de colores para los estados
     color_dict = { 0: "#cc0000", # rojo claro: caja desordenada
                    1: "#b4a7d6", # morado claro: caja ordenada
                    2: "#b6d7a8", # verde claro: montacargas
                    None:'#d5e5d5'}

     # Dibujamos el grid
     ap.gridplot(grid = attr_grid, # arreglo de 2 dimensiones con valores
                 color_dict = color_dict, # diccionario que traduce cada valor de grid a un color
                 ax = ax, # eje para la gráfica
                 convert = True # convierte valores a vectores rgba
               )

     # Etiquetas dinámicas
     ax.set_title(f"Simulacion de montacargas\n"
                  f"Time-step: {model.t}, Cajas ordenadas: "
                  f"{len(model.cajas.select(model.cajas.condicion == 1))}")

In [7]:
fig, ax = plt.subplots() 
model = AlmacenModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=25))